In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.svm import SVC

In [13]:
data = pd.read_csv('CFTrend_data.csv')
data

classes = data['Classification'] 
data = data.drop(['Company', 'Time', 'Classification'], axis = 1)

col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")
    
# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Classification', classes)
data


Net Income
Done
Cash from Operations
Done
Cash from Investing
Done
Cash from Financing
Done
Net Change in Cash
Done
Levered Free Cash Flow
Done
Unlevered Free Cash Flow
Done
Free Cash Flow / Share
Done


,Net Income,Cash from Operations,Cash from Investing,Cash from Financing,Net Change in Cash,Levered Free Cash Flow,Unlevered Free Cash Flow,Free Cash Flow / Share,Classification
0,0.480473,0.469643,0.508359,0.534067,0.606119,0.542375,0.546646,0.437802,0
1,0.481818,0.467858,0.505735,0.537491,0.606840,0.540983,0.545741,0.437786,0
2,0.481596,0.459018,0.491027,0.551337,0.606217,0.531175,0.536074,0.437411,1
3,0.481972,0.469780,0.520974,0.528007,0.610585,0.543266,0.548056,0.437738,1
4,0.480921,0.466917,0.503488,0.538586,0.605890,0.541717,0.546469,0.437794,1
5,0.480900,0.472130,0.502963,0.537991,0.606221,0.542226,0.546985,0.437969,0
6,0.483575,0.446902,0.509085,0.540846,0.605936,0.539428,0.544301,0.437681,1
7,0.483352,0.470616,0.503060,0.537277,0.605155,0.542431,0.547178,0.437757,1
8,0.481091,0.459393,0.503048,0.539351,0.604342,0.538613,0.543393,0.437593,1
9,0.481496,0.472081,0.501630,0.551333,0.618644,0.541420,0.546175,0.437752,1


In [14]:
# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:8] # Parameters
y = NpMatrix[:,8] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))

X: [[0.48047315 0.46964303 0.5083592  ... 0.54237545 0.54664551 0.43780199]
 [0.48181768 0.46785821 0.50573495 ... 0.54098301 0.54574138 0.43778649]
 [0.48159606 0.45901847 0.49102705 ... 0.53117487 0.5360744  0.43741059]
 ...
 [0.47248322 0.38463906 0.47025539 ... 0.56701213 0.57357241 0.43757012]
 [0.48147547 0.46795254 0.49489803 ... 0.54175474 0.5464481  0.43774903]
 [0.50298102 0.46586734 0.50960056 ... 0.54332166 0.54786444 0.43779489]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 8
Total Number of Training instances: 4500


[0. 0. 1. ... 1. 1. 1.]


Type y: <class 'numpy.ndarray'>
Length of Individual y_train vector 0.0
Total number of y values 4500


In [15]:
SVM = SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [16]:
# 15 minutes to run
CV = StratifiedKFold(n_splits=10, random_state=None, shuffle=False)

scores = cross_val_score(SVM, X, y, cv = CV) 

print(scores)

print("Mean Accuracy:", scores.mean())
print("SD of the mean:", scores.std())

[0.67184035 0.68957871 0.68       0.68444444 0.68888889 0.67111111
 0.67555556 0.66666667 0.67260579 0.68374165]
Mean Accuracy: 0.6784433174155593
SD of the mean: 0.007603169532775665
